## Detection for Original Videos
This notebooks walks through model training for frames captured from the original videos

In [1]:
import tensorflow as tf
import numpy as np
from src import ThreeLayerConvNet, Import_Data, optimizer_init_fn

`Import_Data()` is class used to convert images into tensors. 
`get_data` returns X and y tensors for training, testing and validation. `get_labels` returns the image names for the given set

In [2]:
importer = Import_Data()
X_train, y_train, X_test, y_test, X_val, y_val = importer.get_data()

`ThreeLayerConvNet` contains the CNN framework used for training. `optimizer_init_fn` returns a nesterov SGD keras optimizer

In [3]:
model = ThreeLayerConvNet()

This block compiles and trains the network

In [4]:
model.compile(optimizer_init_fn(5e-4),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=16, epochs= 8, validation_data=(X_test, y_test))
#Note we're valdidating on the test set per the task specification
#If satisfactory accuracy isn't reached, increase the number of epochs

Train on 116 samples, validate on 69 samples
Epoch 1/8
116/116 [==============================] - 35s 306ms/sample - loss: 143.5501 - sparse_categorical_accuracy: 0.7241 - val_loss: 28.1815 - val_sparse_categorical_accuracy: 0.8986
Epoch 2/8
116/116 [==============================] - 31s 265ms/sample - loss: 6.5659 - sparse_categorical_accuracy: 0.9397 - val_loss: 0.4700 - val_sparse_categorical_accuracy: 0.9565
Epoch 3/8
116/116 [==============================] - 31s 264ms/sample - loss: 0.6146 - sparse_categorical_accuracy: 0.9569 - val_loss: 0.3133 - val_sparse_categorical_accuracy: 0.9565
Epoch 4/8
116/116 [==============================] - 31s 265ms/sample - loss: 0.1769 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.2314 - val_sparse_categorical_accuracy: 0.9565
Epoch 5/8
116/116 [==============================] - 31s 264ms/sample - loss: 0.0249 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.1088 - val_sparse_categorical_accuracy: 0.9565
Epoch 6/8
116/116 [=============

And this block evaluates how the model does on the validation set and shows the predicted values

In [5]:
loss, acc = model.evaluate(X_val, y_val)
y_val_pred = np.argmax(model.predict(X_val), axis = 1)
print (y_val_pred)

47/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Finally, this block writes the training data labels to `output_original.csv`

In [6]:
import pandas as pd 

def write_output(path,names, y_pred):
    '''
    names must be a list of length N
    y_pred must be a 1D np.array of size N
    '''
    N = y_pred.shape[0]
    pred_word = []
    for i in range(N): 
        if y_pred[i] == 1: 
            pred_word.append('yes')
        else: 
            pred_word.append('no')
    output_df = pd.DataFrame(columns = ['Name','Drinking'])
    output_df['Name'] = names
    output_df['Drinking'] = pred_word
    output_df.to_csv(path, sep = ',', header = False, index = False)
    print ('Output written to %s' % path)
    
y_train_pred = np.argmax(model.predict(X_train), axis = 1)
write_output('output_original.csv',importer.get_labels(), y_train_pred)

Output written to output_original.csv
